### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")

import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime



In [2]:
# 正式
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

# 测试



In [3]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']

order_kargs = {
    "order_status":1,
    "order_time":"yesterday"
}

yd_data = qm.ext_order(order_kargs,sku_goods,free_goods)

order_kargs = {
    "order_status":1,
    "order_time":"today"
}
to_data = qm.ext_order(order_kargs,sku_goods,free_goods)

请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'original', 'page': 1}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 1}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 2}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 3}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 4}
请求 {'order_status': 1, 'order_time': 'today', 'order_id': '', 'order_type': 'original', 'page': 1}
请求 {'order_status': 1, 'order_time': 'today', 'order_id': '', 'order_type': 'free', 'page': 1}


In [4]:
class OrderEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        
        if isinstance(obj, datetime.datetime):  
            return obj.strftime('%Y-%m-%d %H:%M:%S')  
        else:
            return super(MyEncoder, self).default(obj)
def uprow(row,com_set,save_set):

    js = json.dumps(row.to_dict(),cls=OrderEncoder,ensure_ascii=False)

    order_num = row['订单号']
    spp_num = row['发货商ID']
    # 判断是否在已经 发过此物

    if not conn.sismember(com_set ,row['订单号']):
       
        conn.sadd(save_set,row['订单号'])
    else:
        row['订单号'] = None
        
    
    return row

save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)



data = pd.concat([to_data.apply(uprow,axis=1,args=("order:old","order:new_temp")), #
                  yd_data.apply(uprow,axis=1,args=("order:old","order:new_temp"))])
data.dropna(subset=['订单号'],inplace=True)

In [5]:
data

,订单号,订单状态,商品ID,下单时间,支付时间,支付金额,商品名,规格,单位,单价,数量,备注,收件人,联系方式,收货地址,goods_type,发货商ID,订单详情,name
0,3558819475506516,未发货,S0003P0001C0001P01,2019-12-13 15:12:17,2019-12-13 15:12:32,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138.0,1,,熊志林,15874102682,湖南省长沙市芙蓉区湖南省植物保护研究所,free,S0003,三金补水面膜 镇静维稳 深层补水(补水) X 1 = ￥138,ORDER:NEW:S0003
1,9278761409607924,未发货,S0003P0001C0001P01,2019-12-13 15:12:14,2019-12-13 15:12:22,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138.0,1,,汪大鹏,15207442488,湖南省长沙市雨花区巴黎香榭芙蓉公馆(雨花区林子冲路13号),free,S0003,三金补水面膜 镇静维稳 深层补水(补水) X 1 = ￥138,ORDER:NEW:S0003
2,2255178384415585,未发货,S0003P0001C0001P01,2019-12-13 15:12:04,1974-12-30 16:12:50,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138.0,1,,汪大鹏,15207442488,湖南省长沙市雨花区巴黎香榭芙蓉公馆(雨花区林子冲路13号),free,S0003,三金补水面膜 镇静维稳 深层补水(补水) X 1 = ￥138,ORDER:NEW:S0003
3,5198216014617124,未发货,S0003P0001C0002P01,2019-12-13 15:12:19,2019-12-13 15:12:30,0,三金美肤面膜 美白功效 健康肤色(美肤),5片*25g,盒,138.0,1,,姜述维,18173190896,湖南省长沙市开福区茶园路山语城一期(开福区秀峰街道秀峰路69号),free,S0003,三金美肤面膜 美白功效 健康肤色(美肤) X 1 = ￥138,ORDER:NEW:S0003
4,4559036977696915,未发货,S0003P0001C0002P01,2019-12-13 10:12:58,2019-12-13 16:12:14,0,三金美肤面膜 美白功效 健康肤色(美肤),5片*25g,盒,138.0,1,,伍亚男,18900758161,湖南省长沙市岳麓区中海国际社区5期6栋,free,S0003,三金美肤面膜 美白功效 健康肤色(美肤) X 1 = ￥138,ORDER:NEW:S0003
5,9884227472328314,未发货,S0001P0005C0003P01,2019-12-13 15:12:48,2019-12-13 15:12:53,139,AirSolo单耳蓝牙耳机,蓝牙耳机 X 1,副,139.0,1,,汪大鹏,15207442488,湖南省长沙市雨花区巴黎香榭芙蓉公馆(雨花区林子冲路13号),original,NaN,AirSolo单耳蓝牙耳机（规格：蓝牙耳机，商品ID:S0001P0005C0003P01）...,NaN
6,4741213151140980,未发货,S0001P0001C0002P02,2019-12-13 15:12:18,2019-12-13 15:12:20,215,法国大使（Delsey）商务电脑背包,红 X 1,个,215.0,1,,汪大鹏,15207442488,湖南省长沙市雨花区巴黎香榭芙蓉公馆(雨花区林子冲路13号),original,NaN,法国大使（Delsey）商务电脑背包（规格：红，商品ID:S0001P0001C0002P0...,NaN
6,0501250902978832,未发货,S0003P0001C0003P01,2019-12-12 20:12:37,2019-12-12 20:12:46,0,三金护肤面膜 维稳修护 强化屏障(护肤),5片*25g,盒,138.0,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,三金护肤面膜 维稳修护 强化屏障(护肤) X 1 = ￥138,ORDER:NEW:S0003
12,0149891054111246,未发货,S0003P0001C0004P01,2019-12-12 19:12:22,2019-12-12 19:12:32,0,三金水光针(水光针),10ml/支,盒,109.0,1,,杨先生,13307364056,湖南省长沙市岳麓区湖南省长沙市岳麓区湖南涉外经济学院,free,S0003,三金水光针(水光针) X 1 = ￥109,ORDER:NEW:S0003
16,0154427419014946,未发货,S0003P0001C0001P01,2019-12-12 19:12:49,2019-12-12 19:12:02,0,三金补水面膜 镇静维稳 深层补水(补水),5片*25g,盒,138.0,1,,姜女士,18670323312,湖南省长沙市芙蓉区旭日东升b座502,free,S0003,三金补水面膜 镇静维稳 深层补水(补水) X 1 = ￥138,ORDER:NEW:S0003


In [6]:



save(data,"订单")
%run D:\往期\QHJ\ZERO\app\process_order.py


已存入 D:\Downloads\QHJ_MALL\订单 2019-12-13 16_24_37.xlsx
读取供应商信息
读取 D:\Downloads\QHJ_MALL\订单 2019-12-13 16_24_37.xlsx
>>>>系统字段统一为本地字段
>>>>连接供应商信息
存储订外发单信息
>>>>备份

存储新订单 D:\奇货居\work\外发订单\新订单
['订单号', '商品ID', '供应商', '商品名', '数量', '规格', '单位', '收件人', '收货地址', '联系方式', '备注', '发货商']
>>>>正在存储 三金_新订单 2019-12-13 16_24_37.xlsx
>>>>正在存储 傻傻_新订单 2019-12-13 16_24_37.xlsx
新订单存储完成

>>>>打开宏表
>>>>开始运行
>>>>关闭宏表
>>>>关闭完成

3.626s



In [7]:
stop
to_data.apply()

NameError: name 'stop' is not defined

In [ ]:
spp_path =  os.path.join(r"D:\奇货居\素材\商城图片素材","发货商详情.xlsx")
spp_data = pd.read_excel(spp_path)

In [ ]:
spp_data

In [ ]:
stop

In [ ]:
d = dict(zip(spp_data['发货商ID'],spp_data['发货商'],))

d

In [ ]:
conn.keys('ORDER:NEW:TEMP*')

In [ ]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

conn.keys('ORDER:NEW:TEMP*')
src_new = 'ORDER:NEW:TEMP:S0003'
dst_new = src_new.replace(":TEMP","")
conn.sunionstore( dst_new,dst_new,src_new)
conn.delete(src_new)

In [ ]:
# conn.sunionstore('ORDER:NEW',"ORDER:NEW_TEMP",'ORDER:NEW')

# conn.sadd("ORDER:NEW","zero")
# conn.sadd("ORDER:WB","zero")
# conn.sadd("ORDER:NEW_TEMP","zero")